### Review

LLM
- ChatOllama
- ChatOpenAI

Prompt
- PromptTemplate (기본 프롬프트 + invoke 실행과 동시에 변수 삽입)
- ChatPromptTemplate (튜플 형태로 시스템/유저 프롬프트 퓨샷 + invoke 실행과 동시에 변수 삽입)

Response Parser
- OutputParser(기본 Str) 
    - StrOutputParser().invoke(llm.invoke(prompt))
- pydantic 의 BaseModel을 사용하여 Json 구조화된 출력을 생성
    - class CountryDetail(BaseModel):
        capital: str = Field(description="The capital of the country")
        ...
    - llm.with_structured_output(CountryDetail).invoke(prompt.invoke({"country": "France"}))

### 이 모든것들은 Runnable 이며, 체이닝(LCEL)이 가능하다.
- LangChain 은 모두 실행 가능한 것들의 모임.
- | 파이프라인을 통해 실행 결과를 전달할 수 있다.
- 결과를 통해 생긴 체인도 Runnable 이라, 이것들도 체이닝이 가능하다.

### 체이닝 연습
나라, 음식, 레시피 체이닝 연습

In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:1b",
    temperature = 0
)

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

food_prompt = PromptTemplate(
    template= """what is one of the most popular food in {country}?
    Please return the name of the food only.
    """,
    input_variables= ["country"],
)

food_chain = food_prompt | llm | StrOutputParser()

In [5]:
food_chain.invoke({"country": "South Korea"})

'Bibimbap.'

In [12]:
from langchain_core.prompts import ChatPromptTemplate


recipe_prompt = ChatPromptTemplate.from_messages([
    ("system", """Provide the recipe of the food that the user wants.
        Please return recipe only as a numbered list."""),
    ("human", "Can you give me the recipe for making {food}?"),
])

receipe_chain = recipe_prompt | llm | StrOutputParser()

In [13]:
receipe_chain.invoke({"food": "bibimbap"})

"Here's a traditional Korean-style bibimbap recipe:\n\n**Ingredients:**\n\n* 1 cup of mixed vegetables (such as zucchini, carrots, bean sprouts, and spinach)\n* 1/2 cup of cooked white rice\n* 1/4 cup of chopped green onions\n* 1/4 cup of toasted sesame seeds\n* 1/4 cup of crispy garlic, minced\n* 1/2 cup of spicy gochujang sauce (Korean chili paste)\n* 1/4 cup of soy sauce\n* 1/4 cup of rice vinegar\n* 1 tablespoon of Gochugaru (Korean red pepper flakes)\n* 1 teaspoon of ground black pepper\n* 2 eggs, fried or poached\n* Salt to taste\n\n**Instructions:**\n\n1. Cook the mixed vegetables according to your preference.\n2. In a separate pan, heat some oil and fry an egg until it's cooked through.\n3. To assemble the bibimbap, place a scoop of cooked white rice in a bowl.\n4. Arrange the mixed vegetables, fried egg, green onions, sesame seeds, and crispy garlic on top of the rice.\n5. Drizzle the spicy gochujang sauce, soy sauce, rice vinegar, Gochugaru, and black pepper over the ingredie

In [15]:
final_chain = {"food": food_chain} | receipe_chain

In [16]:
final_chain.invoke({"country": "South Korea"})

"Here's a traditional Korean recipe for Bibimbap:\n\n**Servings:** 4-6 people\n\n**Ingredients:**\n\n1. **Rice**: 2 cups of white or brown rice\n2. **Vegetables**: 1 cup of bean sprouts, 1 cup of shredded zucchini, 1/2 cup of diced carrots, 1/2 cup of diced cucumber, 1/4 cup of chopped scallions\n3. **Meat**: 1/2 cup of cooked beef (such as ribeye or sirloin), sliced into thin strips\n4. **Eggs**: 2 eggs, fried sunny-side up\n5. **Sauce**: 2 tablespoons of Gochujang (Korean chili paste)\n6. **Seasonings**: 1 tablespoon of soy sauce, 1 tablespoon of sesame oil, 1 teaspoon of ground black pepper\n\n**Instructions:**\n\n1. Cook the rice according to package instructions.\n2. In a separate pan, heat the Gochujang and sesame oil over medium heat. Add the sliced beef and cook until browned, about 3-4 minutes.\n3. Push the beef to one side of the pan. Crack in the eggs and scramble them until cooked through.\n4. Mix the eggs with the beef.\n5. To assemble the Bibimbap, place a scoop of rice o